In [ ]:
# OR gate problem

### Data

In [36]:
import torch 
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset

In [54]:
dataset_size = 400
train_size = int(dataset_size * 0.5)
test_size = int(dataset_size * 0.5)

In [56]:
x = np.random.normal(0, 0.5, size=(dataset_size, 2))*100
# x

In [57]:
y = np.ones(dataset_size)
# y

In [58]:
y[ x[:,0] * x[:,1] <= 0 ] = 0 
# y

In [59]:
y_train = torch.tensor(y[:train_size], dtype = torch.float32)
x_train = torch.tensor(x[:train_size,:], dtype = torch.float32)

y_valid = torch.tensor(y[train_size:], dtype = torch.float32)
x_valid = torch.tensor(x[train_size:,:], dtype = torch.float32)

In [62]:
td_train = TensorDataset( x_train, y_train ) 

In [63]:
batch_size = 2
dl_train = DataLoader( dataset=td_train, batch_size=batch_size, shuffle=True )

In [67]:
model = nn.Sequential(
    nn.Linear(2,1),
    nn.Softmax()
)

In [68]:
loss_f = nn.BCELoss()
optim  = torch.optim.SGD( lr=0.01, params=model.parameters() ) 

Z:\python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def train( model, x_val, y_val, dl, n_epochs ):
    train_loss = [0] * n_epochs
    train_acc = [0] * n_epochs
    valid_loss = [0] * n_epochs
    valid_acc = [0] * n_epochs
    for e in n_epochs:
        for x_batch, y_batch in dl:
            pred = model(x_batch).squeeze()
            cost = loss_f( pred, y_val )
            cost.backward()
            optim.step()
            optim.zero_grad()
            train_loss[e] += cost.item()
            is_correct = ( pred == y_batch )